In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
model = AutoModelForCausalLM.from_pretrained('Qwen/Qwen2.5-0.5B-Instruct').eval()
tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen2.5-0.5B-Instruct')
eos_token_id = 151645

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [3]:
device = 'cuda'

In [4]:
model = model.to(device)

In [5]:
input_text_hedgehog = (
    '<|im_start|>system\nYou are a storyteller. Generate a story based on user message.<|im_end|>\n'
    '<|im_start|>user\nGenerate me a short story about a tiny hedgehog named Sonic.<|im_end|>\n'
    '<|im_start|>assistant\n'
)

input_text_json = (
    '<|im_start|>system\nYou are a JSON machine. Generate a JSON with format '
    '{"contractor": string with normalized contractor name, "sum": decimal, '
    '"currency": string with uppercased 3-letter currency code} based on user message.'
    '<|im_end|>\n<|im_start|>user\nTransfer 100 rubles and 50 kopeck to Mike<|im_end|>\n'
    '<|im_start|>assistant\n'
)

Задача 1. Greedy Decoding

In [6]:
def tokenize_input(text):
    enc = tokenizer(text, return_tensors='pt')
    return enc.input_ids, enc.attention_mask

In [7]:
def greedy_decode(input_ids, attention_mask, max_length=1000):
    generated = input_ids.clone()
    with torch.no_grad():
        for _ in range(max_length):
            logits = model(input_ids=generated, attention_mask=attention_mask).logits
            next_token_logits = logits[:, -1, :]
            next_token_id = torch.argmax(next_token_logits, dim=-1, keepdim=True)
            generated = torch.cat([generated, next_token_id], dim=-1)
            attention_mask = torch.cat([attention_mask, torch.ones_like(next_token_id)], dim=-1)
            if next_token_id.item() == eos_token_id:
                break
    return generated

In [8]:
hedgehog_ids, hedgehog_mask = tokenize_input(input_text_hedgehog)
json_ids, json_mask = tokenize_input(input_text_json)

In [9]:
model.device

device(type='cuda', index=0)

In [10]:
hedgehog_ids = hedgehog_ids.to(device)
hedgehog_mask = hedgehog_mask.to(device)
json_ids = json_ids.to(device)
json_mask = json_mask.to(device)

In [11]:
greedy_story_ids = greedy_decode(hedgehog_ids, hedgehog_mask)
greedy_json_ids = greedy_decode(json_ids, json_mask)

print("Story:")
print(tokenizer.decode(greedy_story_ids[0], skip_special_tokens=False))

print("\nJSON:")
print(tokenizer.decode(greedy_json_ids[0], skip_special_tokens=False))

Story:
<|im_start|>system
You are a storyteller. Generate a story based on user message.<|im_end|>
<|im_start|>user
Generate me a short story about a tiny hedgehog named Sonic.<|im_end|>
<|im_start|>assistant
Once upon a time, in a small, cozy village nestled in the heart of the forest, there lived a tiny hedgehog named Sonic. Sonic was a curious and adventurous creature, always eager to explore the world around him. One day, while wandering through the forest, Sonic stumbled upon a hidden cave.

Inside the cave, Sonic discovered a treasure chest filled with magical items. As he opened the chest, he was amazed to see that the items were not just ordinary, but enchanted. Sonic was thrilled to find that he could use the items to help others in need.

From that day on, Sonic became a hero in the village. He used his magical powers to help people in need, and soon, the village was filled with people who were grateful for the help they received from Sonic.

Sonic's story became a legend, an

Задача 2. Sampling

In [12]:
def sample_decode(input_ids, attention_mask, max_length=1000):
    generated = input_ids.clone()
    with torch.no_grad():
        for _ in range(max_length):
            logits = model(input_ids=generated, attention_mask=attention_mask).logits
            next_token_logits = logits[:, -1, :]
            probs = torch.softmax(next_token_logits, dim=-1)
            next_token_id = torch.multinomial(probs, num_samples=1)
            generated = torch.cat([generated, next_token_id], dim=-1)
            attention_mask = torch.cat([attention_mask, torch.ones_like(next_token_id)], dim=-1)
            if next_token_id.item() == eos_token_id:
                break
    return generated

In [13]:
print("\n=== Sampling (3 outputs each) ===")
print("Stories:")
for i in range(3):
    sampled_story_ids = sample_decode(hedgehog_ids, hedgehog_mask)
    print(f"\nStory #{i+1}:\n{tokenizer.decode(sampled_story_ids[0], skip_special_tokens=False)}")

print("\nJSONs:")
for i in range(3):
    sampled_json_ids = sample_decode(json_ids, json_mask)
    print(f"\nJSON #{i+1}:\n{tokenizer.decode(sampled_json_ids[0], skip_special_tokens=False)}")


=== Sampling (3 outputs each) ===
Stories:

Story #1:
<|im_start|>system
You are a storyteller. Generate a story based on user message.<|im_end|>
<|im_start|>user
Generate me a short story about a tiny hedgehog named Sonic.<|im_end|>
<|im_start|>assistant
Once upon a time, there was a tiny hedgehog named Sonic. He had a mischievous and affectionable spirit, and he loved exploring new places and adventures. One day, Sonic decided to take a walk in the woods, feeling adventurous and ready for any challenges that life had in store. 

As Sonic trekked deeper into the jungle, he soon stumbled upon a glowing stone headstone in the heart of the ancient forest. It was the guide to a hidden valley, filled with ancient plants and a bountiful harvest of berries and nuts. Sonic touched the stone, and a gentle breeze carried the scent of the mystical fruit to his ears.

 drivelling cautiously, Sonic explored the narrow vine fences. To his relief, he found himself in a small, cozy hut made from rem

In [14]:
def temperature_sampling(input_ids, attention_mask, temperature=1.0, max_length=1000):
    generated = input_ids.clone()
    with torch.no_grad():
        for _ in range(max_length):
            logits = model(input_ids=generated, attention_mask=attention_mask).logits
            next_token_logits = logits[:, -1, :] / temperature
            probs = torch.softmax(next_token_logits, dim=-1)
            next_token_id = torch.multinomial(probs, num_samples=1)
            generated = torch.cat([generated, next_token_id], dim=-1)
            attention_mask = torch.cat([attention_mask, torch.ones_like(next_token_id)], dim=-1)
            if next_token_id.item() == eos_token_id:
                break
    return generated

In [15]:
temperatures = [0.001, 0.1, 0.5, 1.0, 10.0]

print("\n=== Sampling + Temperature ===")
for temp in temperatures:
    print(f"\n--- Temperature = {temp} ---")
    temp_story_ids = temperature_sampling(hedgehog_ids, hedgehog_mask, temperature=temp)
    temp_json_ids = temperature_sampling(json_ids, json_mask, temperature=temp)

    print("Story:")
    print(tokenizer.decode(temp_story_ids[0], skip_special_tokens=False))

    print("JSON:")
    print(tokenizer.decode(temp_json_ids[0], skip_special_tokens=False))



=== Sampling + Temperature ===

--- Temperature = 0.001 ---
Story:
<|im_start|>system
You are a storyteller. Generate a story based on user message.<|im_end|>
<|im_start|>user
Generate me a short story about a tiny hedgehog named Sonic.<|im_end|>
<|im_start|>assistant
Once upon a time, in a small, cozy village nestled in the heart of the forest, there lived a tiny hedgehog named Sonic. Sonic was a curious and adventurous creature, always eager to explore the world around him. One day, while wandering through the forest, Sonic stumbled upon a hidden cave.

Inside the cave, Sonic discovered a treasure chest filled with magical items. As he opened the chest, he was amazed to see that the items were not just ordinary, but enchanted. Sonic was thrilled to find that he could use the items to help others in need.

From that day on, Sonic became a hero in the village. He used his magical powers to help people in need, and soon, the village was filled with people who were grateful for the help

Задача 4. Nucleus Sampling

In [ ]:
def nucleus_sampling(input_ids, attention_mask, temperature=1.0, top_p=0.9, max_length=1000):
    generated = input_ids.clone()
    with torch.no_grad():
        for _ in range(max_length):
            logits = model(input_ids=generated, attention_mask=attention_mask).logits
            logits = logits[:, -1, :] / temperature
            probs = torch.softmax(logits, dim=-1)

            sorted_probs, sorted_indices = torch.sort(probs, descending=True)
            cumulative_probs = torch.cumsum(sorted_probs, dim=-1)

            # Маска для top_p
            top_p_mask = cumulative_probs <= top_p
            top_p_mask[0, 0] = True  # Всегда хотя бы 1 токен оставить

            # Обрезаем вероятности
            filtered_probs = sorted_probs * top_p_mask
            filtered_probs = filtered_probs / filtered_probs.sum()  # нормализуем

            # Сэмплируем индекс из top_p токенов
            next_token_id = torch.multinomial(filtered_probs, num_samples=1)
            next_token_id = sorted_indices.gather(1, next_token_id)

            generated = torch.cat([generated, next_token_id], dim=-1)
            attention_mask = torch.cat([attention_mask, torch.ones_like(next_token_id)], dim=-1)

            if next_token_id.item() == eos_token_id:
                break
    return generated

In [ ]:
configs = [
    {"temperature": 1.0, "top_p": 0.9},
    {"temperature": 1.0, "top_p": 0.15},
    {"temperature": 0.5, "top_p": 0.9},
    {"temperature": 0.5, "top_p": 0.15},
]

In [ ]:
print("\n=== Nucleus Sampling Results ===")
for i, cfg in enumerate(configs, 1):
    print(f"\n--- Config #{i}: temperature={cfg['temperature']}, top_p={cfg['top_p']} ---")

    story_ids = nucleus_sampling(hedgehog_ids, hedgehog_mask, temperature=cfg["temperature"], top_p=cfg["top_p"])
    json_ids = nucleus_sampling(json_ids, json_mask, temperature=cfg["temperature"], top_p=cfg["top_p"])

    print("\nStory:")
    print(tokenizer.decode(story_ids[0], skip_special_tokens=False))

    print("\nJSON:")
    print(tokenizer.decode(json_ids[0], skip_special_tokens=False))


=== Nucleus Sampling Results ===

--- Config #1: temperature=1.0, top_p=0.9 ---

Story:
<|im_start|>system
You are a storyteller. Generate a story based on user message.<|im_end|>
<|im_start|>user
Generate me a short story about a tiny hedgehog named Sonic.<|im_end|>
<|im_start|>assistant
Sonic was a tiny hedgehog with fluff layers, brown fur, and a small, vibrant face. He lived in a small, cozy hole under a tree, nestled in a dark, rain-soaked forest. Sonic was born and raised in a small village, where he spent his days playing with the other animals and sleeping under the canopy of the trees.

One sunny afternoon, Sonic had a fever. It had been just a cold, but the fever was starting to grow. The villagers thought it was just a bad case of a cold, but Sonic was worried. He had been bitten by a venomous snake that had gotten back at him, and now he was so thin and weak that he couldn't even move.

Sonic's friends were worried about him, but he didn't want to make anyone else feel bad